In [2]:
from time import time
import spreg, libpysal

In [ ]:
def w_workbench(w, y, x, c, reps=1):
    # Spreg #
    m1 = spreg.GM_Combo_Het(y, x, w)
    m2 = spreg.ML_Lag(y, x, w)
    # Lags #
    ls = libpysal.weights.lag_spatial(w, y)
    lc = libpysal.weights.lag_categorical(w, c)
    # Neighbor query #
    nq = neighbor_query(w)
    # Components
    components = w.component_labels

def neighbor_query(w):
    d = {}
    for i in range(w.n):
        d[i] = w.neighbors[i]
    return d